$$
u(x,y) = 5(-2x+3y)cos(3x+2y)+3y^2\\
0 \leq x \leq 1,\\
0 \leq y \leq 1\\
$$

$$
u_{xx} = 45 (2 x - 3 y) cos(3 x + 2 y) + 60 sin(3 x + 2 y)\\
u_{yy} = 6 + 20 (2 x - 3 y) cos(3 x + 2 y) - 60 sin(3 x + 2 y)\\
f(x,y) = u_{xx} + u_{yy} = 6 + 65 (2 x - 3 y) cos(3 x + 2 y)\\
$$

_Boundaries:_

$$
\begin{cases}
-u_y(0, y) = \phi_B(y) = -15 cos(2 y) + 6 y (-1 + 5 sin(2 y))
\\
u_y(1, y) = \phi_T(y) = 6 y + 15 cos(3 + 2 y) + (20 - 30 y) sin(3 + 2 y)
\\
-u_x(x, 0) = \phi_L(x) = 10 (cos(3 x) - 3 x sin(3 x))
\\
u_x(x, 1) = \phi_R(x) = -10 cos(2 + 3 x) + 15 (-3 + 2 x) sin(2 + 3 x)
\end{cases}
$$


In [331]:
import numpy as np
from scipy.integrate import dblquad

In [332]:
def sol(x, y):
    return 5*(-2*x+3*y)*np.cos(3*x+2*y)+3*y**2


def f(x, y):
    return 6 + 65*(2*x - 3*y)*np.cos(3*x + 2*y)


def phi_B(y):
    return -15*np.cos(2*y) + 6*y*(-1 + 5*np.sin(2*y))


def phi_T(y):
    return 6*y + 15*np.cos(3 + 2*y) + (20 - 30*y)*np.sin(3 + 2*y)


def phi_L(x):
    return 10*(np.cos(3*x) - 3*x*np.sin(3*x))


def phi_R(x):
    return -10*np.cos(2 + 3*x) + 15*(-3 + 2*x)*np.sin(2 + 3*x)


xmax = 1
ymax = 1
xmin = 0
ymin = 0

dx = 0.5
dy = 0.5

x = np.arange(xmin, xmax + dx, dx)
y = np.arange(ymin, ymax + dy, dy)
print(len(x))

# Matrix for Finite Volumn Method
A = np.zeros(((len(x)-1) * (len(y)-1), (len(x)-1) * (len(y)-1)))
A.shape

3


(4, 4)

In [333]:
class Cell:
    def __init__(self, P, E, W, N, S):
        self.P = P
        self.E = E
        self.W = W
        self.N = N
        self.S = S

    def __repr__(self) -> str:
        return f"(P: {self.P:.2e}, E: {self.E:.2e}, W: {self.W:.2e}, N: {self.N:.2e}, S: {self.S:.2e})"

In [334]:
def get_pos(n, i, j):
    return n * (i - 1) + j - 1


def create_matrix(xmin, ymin, xmax, ymax, dx, dy, f, phi_B, phi_L, phi_R, phi_T):
    x = np.arange(xmin, xmax + dx, dx)
    y = np.arange(ymin, ymax + dy, dy)
    n = len(x) - 1  # minus 1 because we are considering cells, not points
    A = np.zeros(((len(x)-1) * (len(y)-1), (len(x)-1) * (len(y)-1)))
    B = np.zeros((len(x)-1) * (len(y)-1))
    for i in range(1, n+1):
        for j in range(1, n+1):
            B[get_pos(n, i, j)] = dblquad(
                f, x[i-1], x[i-1]+dx, y[j-1], y[j-1]+dy)[0]

    for i in range(n**2-n):
        A[i, i+n] = dx/dy
        A[i+n, i] = dx/dy

    # first row
    A[0, 0] = A[n-1, n-1] = -dy/dx - dx/dy
    B[0] += -dy*phi_L(x[0]+dx/2) - dx*phi_B(y[0]+dy/2)
    B[n-1] += -dy*phi_R(x[n]+dx/2) - dx*phi_B(y[0]+dy/2)
    for i in range(n-1):
        if i > 0:
            A[i, i] = -2*dy/dx - dx/dy
            B[i] += -dx*phi_B(y[0]+dy/2)
        A[i+1, i] = A[i, i+1] = dy/dx

    # middle rows
    for i in range(2, n):
        first_cell_pos = get_pos(n, i, 1)
        last_cell_pos = get_pos(n, i, n)
        B[first_cell_pos] += -dy*phi_L(x[0]+dx/2)
        B[last_cell_pos] += -dy*phi_R(x[n]+dx/2)
        A[first_cell_pos, first_cell_pos] = A[last_cell_pos,
                                              last_cell_pos] = -dy/dx - 2*dx/dy
        for j in range(first_cell_pos, last_cell_pos):
            if j > first_cell_pos:
                A[j, j] = -2*(dx/dy+dy/dx)
            A[j+1, j] = A[j, j+1] = dy/dx

    # last row
    B[get_pos(n, n, 1)] += -dx*phi_T(y[n]+dy/2) - dy*phi_L(x[0]+dx/2)
    B[get_pos(n, n, n)] += -dx*phi_T(y[n]+dy/2) - dy*phi_R(x[n]+dx/2)
    A[n*(n-1), n*(n-1)] = A[n**2-1, n**2-1] = -dy/dx - dx/dy
    for i in range(n*(n-1), n*n-1):
        if i > n*(n-1):
            A[i, i] = -2*dy/dx - dx/dy
            B[i] += dx*phi_T(y[n]+dy/2)
        A[i+1, i] = A[i, i+1] = dy/dx
    return A, B

In [335]:
A, B = create_matrix(xmin, ymin, xmax, ymax, dx, dy,
                     f, phi_B, phi_L, phi_R, phi_T)
# np.savetxt('matrix_A.txt', A, fmt='%.1f')
# np.savetxt('matrix_B.txt', B, fmt='%.2f')
print(A)
print(B)

[[-2.  1.  1.  0.]
 [ 1. -2.  0.  1.]
 [ 1.  0. -2.  1.]
 [ 0.  1.  1. -2.]]
[ 4.77750635 -0.39816532  0.69550613 -2.60073728]


In [336]:
u = np.linalg.solve(A, B.reshape(-1, 1))

n = len(x)-1
u0 = np.zeros(n**2)
for i in range(1, n+1):
    for j in range(1, n+1):
        u0[get_pos(n, i, j)] = sol(x[i]+dx/2, y[j]+dy/2)
print(np.linalg.norm(u-u0))
print(u)

4.4569601385558616e+16
[[-1.11424003e+16]
 [-1.11424003e+16]
 [-1.11424003e+16]
 [-1.11424003e+16]]
